Deep Learning
=============

Assignment 2
------------

Previously in `1_notmnist.ipynb`, we created a pickle with formatted datasets for training, development and testing on the [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html).

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [3]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

First reload the data we generated in `1_notmnist.ipynb`.

In [4]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [5]:
#X_train = train_dataset.reshape(train_dataset.shape[0], -1)
#print(X_train.shape)
#y_train = train_

In [6]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  #print(labels.shape)
  #labels = labels.reshape(labels.shape[0], labels.shape[2])
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


We're first going to train a multinomial logistic regression using simple gradient descent.

TensorFlow works like this:
* First you describe the computation that you want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph. This description is all contained within the block below:

      with graph.as_default():
          ...

* Then you can run the operations on this graph as many times as you want by calling `session.run()`, providing it outputs to fetch from the graph that get returned. This runtime operation is all contained in the block below:

      with tf.Session(graph=graph) as session:
          ...

Let's load all the data into TensorFlow and build the computation graph corresponding to our training:

In [7]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



Let's run this computation and iterate:

In [8]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 15.771458
Training accuracy: 8.8%
Validation accuracy: 11.9%
Loss at step 100: 2.337999
Training accuracy: 71.2%
Validation accuracy: 71.3%
Loss at step 200: 1.908302
Training accuracy: 73.8%
Validation accuracy: 73.8%
Loss at step 300: 1.666944
Training accuracy: 75.2%
Validation accuracy: 74.7%
Loss at step 400: 1.499057
Training accuracy: 75.8%
Validation accuracy: 75.1%
Loss at step 500: 1.371588
Training accuracy: 76.6%
Validation accuracy: 75.4%
Loss at step 600: 1.270197
Training accuracy: 77.0%
Validation accuracy: 75.7%
Loss at step 700: 1.187074
Training accuracy: 77.7%
Validation accuracy: 75.7%
Loss at step 800: 1.117507
Training accuracy: 78.2%
Validation accuracy: 75.7%
Test accuracy: 82.9%


Let's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a `Placeholder` node which will be fed actual data at every call of `session.run()`.

In [9]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run it:

In [10]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    #print('train_labels', train_labels.shape)
    batch_labels = train_labels[offset:(offset + batch_size), :]
    #batch_labels = batch_labels.reshape(batch_size, batch_labels.shape[2])
    #print('batch_data', batch_data.shape)
    #print('batch_labels', batch_labels.shape)
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print(test_labels.shape)
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 12.514751
Minibatch accuracy: 14.1%
Validation accuracy: 14.9%
Minibatch loss at step 500: 1.247874
Minibatch accuracy: 76.6%
Validation accuracy: 75.9%
Minibatch loss at step 1000: 1.667079
Minibatch accuracy: 72.7%
Validation accuracy: 77.0%
Minibatch loss at step 1500: 0.808163
Minibatch accuracy: 80.5%
Validation accuracy: 78.0%
Minibatch loss at step 2000: 1.124071
Minibatch accuracy: 75.8%
Validation accuracy: 77.7%
Minibatch loss at step 2500: 0.714668
Minibatch accuracy: 83.6%
Validation accuracy: 78.5%
Minibatch loss at step 3000: 0.762480
Minibatch accuracy: 81.2%
Validation accuracy: 78.8%
(10000, 10)
Test accuracy: 85.6%


---
Problem
-------

Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units [nn.relu()](https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#relu) and 1024 hidden nodes. This model should improve your validation / test accuracy.

---

In [93]:
print('train_dataset', train_dataset.shape)
print('train_labels', train_labels.shape)
print('valid_dataset', valid_dataset.shape)
print('valid_labels', valid_labels.shape)
print('test_dataset', test_dataset.shape)
print('test_labels', test_labels.shape)
dims = train_dataset.shape[1]
hidden_layers = 128
num_classes = 10
print(dims)
batch_size = 128
graph1 = tf.Graph()
with graph1.as_default():
    X = tf.placeholder(tf.float32, [None, dims])
    y_labels = tf.placeholder(tf.float32, [None, num_classes])
    
    W1 = tf.Variable(tf.random_uniform([784, hidden_layers], -0.01, 0.01), dtype=tf.float32)
    b1 = tf.Variable(tf.zeros([hidden_layers]), dtype=tf.float32)

    z1 = tf.matmul(X, W1) + b1
    a1 = tf.nn.relu(z1)
    
    W2 = tf.Variable(tf.random_uniform([hidden_layers, num_classes], -0.01, 0.01), dtype=tf.float32)
    b2 = tf.Variable(tf.zeros([num_classes]), dtype=tf.float32)
    
    z2 = tf.matmul(a1, W2) + b2
    a2 = tf.nn.softmax(z2)
    
    loss1 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_labels, logits=a2))
    optimizer = tf.train.GradientDescentOptimizer(0.2).minimize(loss1)
    
    with tf.Session(graph=graph1) as sess:
        l_value = None
        
        tf.global_variables_initializer().run()
        for i in range(200000):
            offset = (i * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            #print('train_labels', train_labels.shape)
            batch_labels = train_labels[offset:(offset + batch_size), :]
            feed_dict={X:batch_data, y_labels:batch_labels}
            _, l_value = sess.run([optimizer, loss1], feed_dict=feed_dict)
            if i % 100 == 0:
                print(l_value)
        print("Final Loss", l_value)

        correct_prediction = tf.equal(tf.argmax(y_labels,1), tf.argmax(a2,1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        print(sess.run(accuracy, feed_dict={X: valid_dataset, y_labels:valid_labels}))

train_dataset (200000, 784)
train_labels (200000, 10)
valid_dataset (10000, 784)
valid_labels (10000, 10)
test_dataset (10000, 784)
test_labels (10000, 10)
784
2.3026755
2.299971
2.249751
1.9950615
1.7240877
1.6771815
1.6877968
1.6559422
1.6902298
1.663902
1.6548221
1.593581
1.6790941
1.7198652
1.6137357
1.599839
1.673797
1.57812
1.600399
1.6442189
1.6315404
1.675123
1.6508974
1.6513009
1.6110241
1.5885293
1.6460965
1.6473707
1.6307552
1.6168915
1.6188986
1.6691668
1.6232721
1.5814309
1.6284317
1.6115577
1.6100081
1.6241571
1.6186278
1.6362827
1.698436
1.595495
1.6448457
1.6497885
1.6130058
1.6888382
1.6139014
1.5781016
1.6374394
1.6466752
1.5905969
1.632684
1.6502852
1.5803154
1.6076884
1.6023113
1.5707619
1.6565435
1.6323323
1.6195722
1.5934622
1.5897868
1.6046535
1.6043689
1.6244893
1.6131513
1.553451
1.5896659
1.5930607
1.5998042
1.6225529
1.5812749
1.6071013
1.6133237
1.5487944
1.6482971
1.5864351
1.6055639
1.6143161
1.6146524
1.6206725
1.5911314
1.6160393
1.6167624
1.6184583
1.60

1.5291262
1.5306764
1.5403614
1.5570714
1.5488048
1.5342666
1.5513808
1.5703065
1.563541
1.5076163
1.5633258
1.5571935
1.5404046
1.5930743
1.5122831
1.6048441
1.5240662
1.5563096
1.5431161
1.5868503
1.5087581
1.5515168
1.5250332
1.5261362
1.5403817
1.5660431
1.5518188
1.5767857
1.5267969
1.5297796
1.5344726
1.5315088
1.5241585
1.5252218
1.5581574
1.5698448
1.5548873
1.5420008
1.5644816
1.6093698
1.5562706
1.5580931
1.5458132
1.5860837
1.558912
1.5571437
1.5500853
1.543258
1.5273951
1.5636879
1.5849767
1.5331403
1.538778
1.5637977
1.5779305
1.5707061
1.5542271
1.5248165
1.5260742
1.5795896
1.529251
1.5325128
1.5518911
1.5493755
1.5383663
1.5399946
1.5316517
1.5312115
1.4772351
1.5125971
1.5416605
1.5420785
1.5621858
1.5865982
1.541157
1.5047452
1.5272903
1.5901711
1.519698
1.5476339
1.5302228
1.59284
1.5187724
1.5735328
1.5473192
1.5107653
1.5424359
1.5247879
1.5640752
1.5180135
1.5404621
1.5265932
1.5850484
1.5642676
1.5576236
1.5452629
1.5488966
1.502407
1.551956
1.5411378
1.5455737
1

1.5386444
1.5853403
1.5313632
1.5243461
1.5467696
1.5319602
1.546952
1.5468438
1.584771
1.5480878
1.5322244
1.5355775
1.5001338
1.5244077
1.5400655
1.5643816
1.5251486
1.5308982
1.5172901
1.5165694
1.5658103
1.5120487
1.5434579
1.5354356
1.5220238
1.5330111
1.5694228
1.5544477
1.6007663
1.5185013
1.5781257
1.6030668
1.5592314
1.5806361
1.5543435
1.5770446
1.5670295
1.4929873
1.5166714
1.509584
1.5548903
1.5503478
1.554772
1.5541539
1.5389282
1.5159397
1.56475
1.5177481
1.5009081
1.5602125
1.5405366
1.5262705
1.5610733
1.540469
1.5009108
1.5479565
1.5322864
1.5420685
1.5343554
1.5087647
1.547745
1.5611529
1.5214684
1.5551549
1.530746
1.5320666
1.5393484
1.600623
1.5176225
1.5340018
1.542278
1.5396975
1.518875
1.5463352
1.5624841
1.5452096
1.5393949
1.547974
1.5395663
1.539928
1.5168296
1.5628928
1.5621146
1.5636368
1.5389221
1.524224
1.525409
1.5239348
1.507882
1.5471513
1.5546329
1.5221877
1.56878
1.4857018
1.5694711
1.5162959
1.54041
1.5777929
1.5243769
1.5331106
1.5392845
1.5387992
1